# food-manufacture-i.py

notebook 版本

## Food Manufacture I Example
Source: http://www.gurobi.com/resources/examples/food-manufacture-I

Both this model and Food Manufacture II are examples of a blending problem. In blending optimization problems, multiple raw materials are combined in a way the meets the stated constraints for the lowest cost. These problems are common in numerous industries including, for example, the oil industry (blending different types of crude oil at a refinery) and agriculture (manufacturing feed that meets the different nutritional requirements of different animals).

In this particular example, we’ll model and solve a production planning problem where we must create a final product from a number of ingredients, each of which has different costs, restrictions and features. The aim is to create an optimal multi-period production plan that maximizes profit. More details can be found on the Problem Description and Model Formulation tabs below.

In Food Manufacture II, we'll extend this example with additional constraints that change the problem type from a linear program (LP) to a mixed integer program (MIP), which makes it harder to solve.

Note: you can download the model, implemented in Python, here. More information on this type of model can be found in the fifth edition of Model Building in Mathematical Programming, by H. Paul Williams.

Reference: H. Paul Williams, Model Building in Mathematical Programming, fifth edition (Page 253-254, 296-298, 349-350)

version 1

## Problem Description
A manufacturer needs to refine (提炼) several raw oils and blend (混合) them together to produce a given food product that can be sold. The raw oils needed can be divided into two categories:

Category|	Oil|
--------|------|-----
Vegetable oils:	|VEG 1
 	|VEG 2
Non-vegetable oils:	|OIL 1
 	|OIL2
 	|OIL 3
    
The manufacturer can choose to buy raw oils for the current month and/or buy them on the futures market for delivery in a subsequent month. Prices for immediate deliver and in the futures market are given below in $/ton: 生产商每月购买原料的价格表

Month|	VEG 1|	VEG 2|	OIL 1|	OIL 2|	OIL 3|
-----|--------|------|-------|-------|-------|
January|	110|	120|	130|	110|	115
February|	130|	130|	110|	90|	115
March	|110	|140	|130	|100	|95
April	|120	|110	|120	|120	|125
May    |100	|120	|150	|110	|105
June   |90	    |100	|140	|80	    |135

There are a number of additional factors that must be taken into account. These include:

1. The final food product sells for \$150 per ton.  最终产品售价150元/吨
1. Each category of oil (vegetable and non-vegetable) needs to be refined on a different production line.  不同类型油在不同生产线生产
1. There is limited refinement capacity such that in any given month a maximum of 200 tons of vegetable oils and 250 tons of non-vegetable oils can be refined.  有200(negetable) /250 (non-vegetable) 吨的生产限额
1. Also, there is no waste in the refinement process, so the sum of the raw oils refined will equal the amount of refined oils available.  提炼没有损失(理想化)
1. The cost of refining the oils may be ignored.  提炼不需要额外花费(理想化)

In addition to the refining limits above, there are limits to the amount of raw oils that can be stored for future use, and there is a cost associated with each ton of oil stored. The limit is 1000 tons of each raw oil and the storage cost is \$5 per ton per month. The manufacturer can not store the produced food product or the refined oils.   每种油存量不能超过 1000吨,并且有 \$5/每吨的仓储费用.

The final food product must have a hardness between three and six on a given hardness scale. For the purposes of the model, hardness blends linearly and the hardness of each raw oil is: 混合之后的 Hardness 在 3-6之间

Oils|	Hardness
----|-----------
VEG 1|	8.8
VEG 2|	6.1
OIL 1|	2.0
OIL 2 |	4.2
OIL 3|	5.0

At the start of January there are 500 tons of each type of raw oil in storage. For the purpose of the model, this should also be the level of raw oils in storage at the end of June. 1月初和6月末有500吨油在仓库中

Given the above information, what monthly buying and manufacturing decisions should be made in order to maximize profit? 给出每月的购买和生产计划

This problem is based on a larger model built for the margarine producer Van den Bergs and Jurgens and discussed in Williams and Redwood (1974).

### Step 1: Import functions and create Constants

In [1]:
from gurobipy import *


In [2]:



time_periods = ["January", "February", "March", "April", "May", "June"]

oils = ["VEG1", "VEG2", "OIL1", "OIL2", "OIL3"]

prices = tupledict({
	('January', 'VEG1'): 110,
	('January', 'VEG2'): 120,
	('January', 'OIL1'): 130,
	('January', 'OIL2'): 110,
	('January', 'OIL3'): 115,
	('February', 'VEG1'): 130,
	('February', 'VEG2'): 130,
	('February', 'OIL1'): 110,
	('February', 'OIL2'): 90,
	('February', 'OIL3'): 115,
	('March', 'VEG1'): 110,
	('March', 'VEG2'): 140,
	('March', 'OIL1'): 130,
	('March', 'OIL2'): 100,
	('March', 'OIL3'): 95,
	('April', 'VEG1'): 120,
	('April', 'VEG2'): 110,
	('April', 'OIL1'): 120,
	('April', 'OIL2'): 120,
	('April', 'OIL3'): 125,
	('May', 'VEG1'): 100,
	('May', 'VEG2'): 120,
	('May', 'OIL1'): 150,
	('May', 'OIL2'): 110,
	('May', 'OIL3'): 105,
	('June', 'VEG1'): 90,
	('June', 'VEG2'): 100,
	('June', 'OIL1'): 140,
	('June', 'OIL2'): 80,
	('June', 'OIL3'): 135
})


hardness = {"VEG1": 8.8, "VEG2": 6.1, "OIL1": 2.0, "OIL2": 4.2, "OIL3": 5.0}

price = 150
IStore = 500
vegCapa = 200
oilCapa = 250

hardness_lb = 3
hardness_ub = 6
store_pricing = 5



### Step 2: Create  model

In [3]:
model = Model('Food Manufacture I')

### Step 3: Create activitiy variables

In [4]:
# 生产数量 Quantity of food produced in each period # 简单数值的 vars 可以使用  food.sum()
food = model.addVars(time_periods, name = "Food")
# 提炼refine 数量 Quantity used of each product  in each period  
use = model.addVars(time_periods, oils, name = "Use")
# Quantity stored of each product  in each period  # 简单数值的 vars 可以使用  store.sum()
store = model.addVars(time_periods, oils, name = "Store")

# 购买原材料 Quantity bought of each product in each period # 生产 Buy 矩阵, 预备和 prices 进行矩阵相乘 buy.prodct(prices)
buy = model.addVars( time_periods, oils , name = "Buy")



### Step 4: Set objective function

In [5]:

# Objective
obj = price*food.sum() - buy.prod(prices) - store_pricing*store.sum()
model.setObjective(obj, GRB.MAXIMIZE) # maximize profit

# 我们的写法
# 设定约束: capa 限制 200/250
# for t in time_periods:
#     model.addConstr(quicksum([store[t, o] for o in oils if (o == "VEG1" or o == "VEG2")] ) ==vegCapa)
#     model.addConstr(quicksum([store[t, o] for o in oils if (o == "OIL1" or o == "OIL2" or o == "OIL3")] ) ==oilCapa)

# 设定约束: Buy - Use == Store
#model.addConstrs(buy[t, o] - use[t, o] == store[t, o] for t in time_periods for o in oils)

### Step 5: Add constraints

In [6]:

#Initial Balance
model.addConstrs((IStore + buy[time_periods[0], oil] ==
     use[time_periods[0], oil] + store[time_periods[0], oil] for oil in oils), "Initial_Balance")

#Balance
model.addConstrs(
	(store[time_periods[time_periods.index(time_period)-1], oil] + buy[time_period, oil] ==
     use[time_period, oil] + store[time_period, oil]
     for oil in oils for time_period in time_periods if time_period != time_periods[0]), "Balance")

#End Store
model.addConstrs((store[time_periods[-1], oil] == IStore for oil in oils),
				 "End_Balance")

# Capacity1 & Capacity2
model.addConstrs(
	(quicksum(use[time_period, oil] for oil in oils if "VEG" in oil) <= vegCapa
	 for time_period in time_periods), "Capacity_Veg")
model.addConstrs(
	(quicksum(use[time_period, oil] for oil in oils if "OIL" in oil) <= oilCapa
	 for time_period in time_periods), "Capacity_Oil")

# Hardness
model.addConstrs(
	(quicksum(hardness[oil] * use[time_period, oil] for oil in oils)
	 >= hardness_lb * food[time_period] for time_period in time_periods),
	"Hardness_lower")
model.addConstrs(
	(quicksum(hardness[oil] * use[time_period, oil] for oil in oils)
	 <= hardness_ub * food[time_period] for time_period in time_periods),
	"Hardness_upper")

# Conserve
model.addConstrs((use.sum(time_period) == food[time_period]
				  for time_period in time_periods), "Conserve")


{'April': <gurobi.Constr *Awaiting Model Update*>,
 'February': <gurobi.Constr *Awaiting Model Update*>,
 'January': <gurobi.Constr *Awaiting Model Update*>,
 'June': <gurobi.Constr *Awaiting Model Update*>,
 'March': <gurobi.Constr *Awaiting Model Update*>,
 'May': <gurobi.Constr *Awaiting Model Update*>}

### Step 6: Solve model

In [7]:
# The objective is to max the profit
obj = price * food.sum() - buy.prod(prices) - store_pricing * store.sum()
model.setObjective(obj)

In [8]:
model.optimize()

Optimize a model with 65 rows, 96 columns and 258 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [5e+00, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 5e+02]
Presolve removed 28 rows and 45 columns
Presolve time: 0.02s
Presolved: 37 rows, 51 columns, 149 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.5261281e+05   9.097405e+03   0.000000e+00      0s
      36    1.0784259e+05   0.000000e+00   0.000000e+00      0s

Solved in 36 iterations and 0.03 seconds
Optimal objective  1.078425926e+05


### Step 8: Print variable values for optimal solution

In [9]:
# Display solution (print the name of each variable and the solution value)
for v in model.getVars():
    if v.X != 0:
        print("%s %f" % (v.Varname, v.X))

Food[January] 450.000000
Food[February] 450.000000
Food[March] 450.000000
Food[April] 450.000000
Food[May] 450.000000
Food[June] 450.000000
Use[January,VEG2] 200.000000
Use[January,OIL3] 250.000000
Use[February,VEG1] 159.259259
Use[February,VEG2] 40.740741
Use[February,OIL2] 250.000000
Use[March,VEG1] 159.259259
Use[March,VEG2] 40.740741
Use[March,OIL2] 250.000000
Use[April,VEG1] 159.259259
Use[April,VEG2] 40.740741
Use[April,OIL2] 250.000000
Use[May,VEG1] 22.222222
Use[May,VEG2] 177.777778
Use[May,OIL2] 250.000000
Use[June,VEG1] 159.259259
Use[June,VEG2] 40.740741
Use[June,OIL2] 250.000000
Store[January,VEG1] 500.000000
Store[January,VEG2] 300.000000
Store[January,OIL1] 500.000000
Store[January,OIL2] 500.000000
Store[January,OIL3] 250.000000
Store[February,VEG1] 340.740741
Store[February,VEG2] 259.259259
Store[February,OIL1] 500.000000
Store[February,OIL2] 750.000000
Store[February,OIL3] 250.000000
Store[March,VEG1] 181.481481
Store[March,VEG2] 218.518519
Store[March,OIL1] 500.000000
